In [1]:
import pandas as pd
from siuba import *

In [2]:
df = pd.read_parquet('_rt_progress_2024-11-13.parquet')

In [18]:
df >> filter(_.organization_source_record_id == 'recPnGkwdpnr8jmHB')

,organization_name,organization_source_record_id,name,base64_url,caltrans_district,status,analysis_date
1,Los Angeles County Metropolitan Transportation...,recPnGkwdpnr8jmHB,LA Metro Bus Schedule,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,07 - Los Angeles,speedmap_segs_available,2024-11-13
2,Los Angeles County Metropolitan Transportation...,recPnGkwdpnr8jmHB,LA Metro Rail Schedule,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaW...,07 - Los Angeles,speedmap_segs_available,2024-11-13


In [17]:
df >> filter(_.organization_source_record_id == 'recuGkFhN2WXGK67H')

,organization_name,organization_source_record_id,name,base64_url,caltrans_district,status,analysis_date
95,City of Banning,recuGkFhN2WXGK67H,Banning Pass Schedule,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,08 - San Bernardino,speedmap_segs_available,2024-11-13


In [6]:
from shared_utils import rt_dates, rt_utils, catalog_utils, schedule_rt_utils
import geopandas as gpd

In [8]:
catalog = catalog_utils.get_catalog('gtfs_analytics_data')

In [11]:
from update_vars_index import ANALYSIS_DATE

In [43]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{ANALYSIS_DATE}.parquet'
speedmap_segs = gpd.read_parquet(path)

In [47]:
assert (speedmap_segs >> select(-_.route_short_name)).isna().any().any() == False, 'no cols besides route_short_name should be nan'

In [26]:
catalog.gcs_paths.keys()

dict_keys(['GCS', 'COMPILED_CACHED_VIEWS', 'SEGMENT_GCS', 'SCHED_GCS', 'RT_SCHED_GCS', 'SHARED_GCS', 'PREDICTIONS_GCS', 'PUBLIC_GCS'])

In [28]:
xpath = f"{catalog.gcs_paths.SCHED_GCS}{catalog['schedule_tables']['gtfs_key_crosswalk']}_{ANALYSIS_DATE}.parquet"

In [29]:
xpath

'gs://calitp-analytics-data/data-analyses/gtfs_schedule/crosswalk/gtfs_key_organization_2024-11-13.parquet'

In [36]:
df2 = pd.read_parquet(xpath)

In [37]:
df2 >> head(3)

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,organization_source_record_id,organization_name,caltrans_district,counties_served,hq_city,hq_county,...,service_area_sq_miles,population,service_area_pop,subrecipient_type,primary_uza,reporter_type,organization_type,voms_pt,voms_do,year
0,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,reckQmUdXUzHFmlVf,City of Ojai,07 - Los Angeles,None,Ojai,None,...,<NA>,NaN,<NA>,Rural General Public Transit,None,Rural Reporter,County or Local Government Unit or Department ...,<NA>,2,2022
1,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,rec7EN71rsZxDFxZd,Ventura County Transportation Commission,07 - Los Angeles,Santa Clara;Ventura,Camarillo,Ventura,...,28,376117.0,209877,None,None,Full Reporter,Public Agency or Authority of Transit Service,45,<NA>,2022
2,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,recS7GnKTcQVX20HE,Gold Coast Transit District,07 - Los Angeles,Ventura,Oxnard,Ventura,...,84,376117.0,374827,None,None,Full Reporter,Public Agency or Authority of Transit Service,22,49,2022


In [32]:
df2 >> filter(_.caltrans_district.isna()) >> distinct(_.name, _keep_all=True)

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,organization_source_record_id,organization_name,caltrans_district,counties_served,hq_city,hq_county,...,service_area_sq_miles,population,service_area_pop,subrecipient_type,primary_uza,reporter_type,organization_type,voms_pt,voms_do,year


In [38]:
CROSSWALK_COLS = [
    "schedule_gtfs_dataset_key", "name",
    "caltrans_district",
    "organization_source_record_id", "organization_name",
    "base64_url"
]

In [40]:
df2[CROSSWALK_COLS].drop_duplicates()

,schedule_gtfs_dataset_key,name,caltrans_district,organization_source_record_id,organization_name,base64_url
0,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,07 - Los Angeles,reckQmUdXUzHFmlVf,City of Ojai,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz
1,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,07 - Los Angeles,rec7EN71rsZxDFxZd,Ventura County Transportation Commission,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz
2,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,07 - Los Angeles,recS7GnKTcQVX20HE,Gold Coast Transit District,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz
3,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,07 - Los Angeles,rec1ErIn9gG1Isk5W,City of Simi Valley,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz
4,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,07 - Los Angeles,recojKzQsBzE1hjVu,City of Moorpark,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz
...,...,...,...,...,...,...
218,7f2ae5bcb719c5912462858471a28aa5,Glenn Schedule,03 - Marysville,rec0sMQyK2v8Cs4Io,Glenn County,aHR0cHM6Ly9jZG4uaGlidXdlYnNpdGVzLmNvbS9hZjI2ZG...
219,ff1bc5dde661d62c877165421e9ca257,Santa Ynez Mecatran Schedule,05 - San Luis Obispo,reckp33bhAuZlmO1M,City of Solvang,aHR0cDovL2FwcC5tZWNhdHJhbi5jb20vdXJiL3dzL2ZlZW...
220,b475cf3b7ceaae9fc30b22cb8d3a1e33,SLO Schedule,05 - San Luis Obispo,recMM99msxjmc6PPv,City of San Luis Obispo,aHR0cHM6Ly9hcHAubWVjYXRyYW4uY29tL3VyYi93cy9mZW...
221,c4092405159366c705b62df938293a4e,Metrolink Schedule,08 - San Bernardino,recQp6uQAUFcenmQ6,San Bernardino County Transportation Authority,aHR0cHM6Ly93d3cubWV0cm9saW5rdHJhaW5zLmNvbS9nbG...


In [19]:
df >> filter(_.caltrans_district.isna()) >> distinct(_.name, _keep_all=True)

,organization_name,organization_source_record_id,name,base64_url,caltrans_district,status,analysis_date


In [16]:
speedmap_segs >> filter(_.caltrans_district.isna()) >> distinct(_.name, _keep_all=True)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,direction_id,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch,route_short_name
0,015d67d5b75b5cf2b710bbadadfb75f5,9de0051e42d628ca7e100bb7c66ebe95,47,35,1.0,40414__40467,2nd St & Grand Ave__Bellam Blvd & Francisco Bl...,40414-40467-1,AM Peak,23.25,...,25.49,Bay Area 511 Marin Schedule,None,recNOb7pqBRlQVG5e,Marin County Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.51997 37.97029, -122.51996 37...",5,1.666667,35
1,0666caf3ec1ecc96b74f4477ee4bc939,feb78ef0323d9eb3bf79f5fa326a0f12,7201312_JUNE24,720-13183,0.0,3033__3120,5th / Colorado__5th / Santa Monica,3033-3120-1,AM Peak,9.25,...,11.58,LA Metro Bus Schedule,None,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (-118.49124 34.01468, -118.49124 34...",15,5.000000,720
2,0881af3822466784992a49f1cc57d38f,6c0150fab06fb29eccfe9c72e1143384,p_897680,SMART,1.0,71021__71031,San Rafael SMART__Marin Civic Center SMART,71021-71031-1,AM Peak,25.80,...,28.12,Bay Area 511 Sonoma-Marin Area Rail Transit Sc...,None,recUmm4gcNXaqrwpn,Sonoma-Marin Area Rail Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.52271 37.97198, -122.52270 37...",5,1.666667,SMART
3,09a703757d1ed14ca9580b1385e39315,a1c3d1dab32a346fd7d575250cbb41c3,21872,6117,0.0,8768759__8768760,Hawthorne Blvd & Marine Ave__Hawthorne Blvd & ...,8768759-8768760-1,AM Peak,20.27,...,26.35,Lawndale Beat GMV Schedule,None,recj8LXdeSurpSRNU,City of Lawndale,aHR0cHM6Ly9yaWRlbGF3bmRhbGViZWF0LmNvbS9ndGZz,"LINESTRING (-118.35233 33.89510, -118.35237 33...",4,1.333333,EX
4,09e16227fc42c4fe90204a9d11581034,1c56e305a58b51f30f3a46ae5f2a93c9,p_900750,48,1.0,7715710__7715720,Range Ave & Steele Ln__Range Ave & Paulin Creek,7715710-7715720-1,AM Peak,28.73,...,36.02,Bay Area 511 Sonoma County Transit Schedule,None,recDupUxInMUgxeiz,Sonoma County,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.73249 38.46077, -122.73249 38...",2,0.666667,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,f4c3ea214214ee0d96f7646b3e9d69dc,12c175740e7def3747210f21da4e81bb,10668,11790,1.0,122006__121951,Pismo at Archer__Higuera at South (SB),122006-121951-1,AM Peak,20.61,...,20.61,SLO Peak Transit Schedule,None,recMM99msxjmc6PPv,City of San Luis Obispo,aHR0cDovL2RhdGEucGVha3RyYW5zaXQuY29tL3N0YXRpY2...,"LINESTRING (-120.66787 35.27334, -120.66801 35...",3,1.000000,2B
82,f5a749dd65924e025b1293c58f95f8d6,ad12bceb4dfbf8ae7788bb65472f9431,5cf6811a-2f53-4199-b315-4408eb816e82,CC,0.0,74742__74689,Richmond__Martinez,74742-74689-1,PM Peak,37.28,...,37.28,Bay Area 511 Capitol Corridor Schedule,None,recvEBkSBc7UxlarC,Capitol Corridor Joint Powers Authority,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.35390 37.93677, -122.35432 37...",3,0.600000,CC
83,f74424acf8c41e4c1e9fd42838c4875c,ba7e1331e1a0c7139ed0c736aeaed04e,18106_shp,194,0.0,2224__2233,Ramona Blvd and Lexington Ave E__Ramona Blvd a...,2224-2233-1,AM Peak,27.64,...,27.64,Foothill Schedule,None,recZm8PD8WIdcDL0M,City of Duarte,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,"LINESTRING (-118.03970 34.07202, -118.03948 34...",10,3.333333,194
84,fb467982dcc77a7f9199bebe709bb700,42e6a69955af03151dab76b173c49f4e,115141,53,0.0,61339__61340,Washington & Charles__Washington & Pastoria,61339-61340-1,AM Peak,17.50,...,17.74,Bay Area 511 Santa Clara Transit Schedule,None,recC5CT95EufmQCXr,Santa Clara Valley Transportation Authority,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.03542 37.37704, -122.03648 37...",5,1.666667,53
